# Bang-Bang 
In this document we will handle moving the vehicle according to the basic Lidar unit.

### What is Lidar 
There are varieties that vary according to the lidar property, rotating 360 degrees and looking fixed at a certain angle. The 360 degrees we use in our car, and each turn gives us the distances around it in the form of an arraylist.


Here is the turning of the vehicle to the right and left according to the distance of the right and left objects (in this example wall). If there is an object to the left, the right angle will be sent to the right if there is an object to the left.

In [ ]:
#!/usr/bin/env python


import rospy
import math
import numpy


from ackermann_msgs.msg import AckermannDriveStamped # Engine Messages
from sensor_msgs.msg import LaserScan, Joy # Joystick and Lidar Messages
class WallE():

    angle=0
    
    def getError(self,goal,L,begin,end):
        return goal-(min(L[begin:end]))
    
    
    def getSteeringCmd(self,error,threshold,fullLeft,fullRight):
        if(abs(error)<threshold):
            return 0
        elif error>0:
            return fullLeft
        else:
            return fullRight
    

    def callback(self,msg):
        #Get Lidar Information
        error=self.getError(.4, msg.ranges, 540, 930)
        self.angle=self.getSteeringCmd(error, .03, -1, 1)
        
    def shutdown(self):
        rospy.loginfo("Car is Stopped ")
        
        self.drive.publish(AckermannDriveStamped())
        rospy.sleep(1)
    
    def __init__(self):
        
        rospy.init_node('wall_bang', anonymous=False)
        rospy.on_shutdown(self.shutdown)
        
        
        rate = 10 
        r = rospy.Rate(rate)
        
        
        self.drive = rospy.Publisher('/vesc/ackermann_cmd_mux/input/navigation', AckermannDriveStamped, queue_size=5)
        
        
        rospy.Subscriber('scan', LaserScan, self.callback)
        
    
        speed = 0.5 # fixed speed rate in meters / second
        dist_trav = 20.0 # way to go
        
        #I've written the fields in the steering message of ackermann (to go straight)
        drive_cmd = AckermannDriveStamped()
        drive_cmd.drive.speed = speed
        drive_cmd.drive.steering_angle = self.angle
      
        
        #the main transaction cycle (works as far as the path to go)
        time = dist_trav / speed
        ticks = int(time * rate) 
        for t in range(ticks):
        #while True: The vehicle can be used for continuous operation, not for constant distance.
            

            drive_cmd.drive.steering_angle=self.angle
            
            self.drive.publish(drive_cmd) #we send the motor what to do.
            r.sleep() 
       
        self.drive.publish(AckermannDriveStamped())
        
if __name__ == '__main__':

    WallE()


Run this application by editing it according to the lidar in your car.